# Example- Zero-Knowledge Proof of File Possession

An investigator possesses a file $F$ and aims to prove ownership without revealing its content. This can be achieved using a Zero-Knowledge Proof (ZKP) based on cryptographic commitments.

---

## Workflow

1. **Hash the File**: Compute the hash of the file, $h = H(F)$, using a secure cryptographic hash function.
2. **Generate Commitment**: 
    - Choose a random nonce $r$.
    - Compute the commitment $t = g^r \mod p$, where $g$ is a generator of the group.
3. **Derive Challenge**: 
    - Combine the file hash $h$ and the commitment $t$ to derive a challenge $c$ (e.g., using the Fiat-Shamir heuristic).
4. **Compute Response**: 
    - Respond with $s = (r + c \cdot x) \mod q$, where $x$ is the secret derived from $h$.
5. **Verification**: 
    - Verify the proof by checking if $g^s \mod p = t \cdot y^c \mod p$, where $y = g^x \mod p$ is the public value.

---

## Demonstration


In [28]:
import random

def generate_parameters():
    while True:
        # Generate a random prime p
        p = random.randint(1000, 10000)
        if all(p % i != 0 for i in range(2, int(p**0.5) + 1)):
            # Find a prime q such that q divides p-1
            for q in range(2, p):
                if (p - 1) % q == 0 and all(q % i != 0 for i in range(2, int(q**0.5) + 1)):
                    # Find a generator g for the subgroup of order q
                    for g in range(2, p):
                        if pow(g, q, p) == 1 and pow(g, (p-1)//q, p) != 1:
                            return p, q, g

def keygen(p, q, g):
    x = random.randint(1, q-1)  # Prover's secret
    y = pow(g, x, p)            # Public key
    return x, y

In [ ]:
import hashlib

def schnorr_proof_of_possession(file_content, p, q, g):
    # Step 1: Prover computes the hash of the file
    h = int.from_bytes(hashlib.sha256(file_content).digest(), byteorder='big') % q
    x = h  # Treat the hash as the secret x for proof

    # Step 2: Compute the public value
    y = pow(g, x, p)

    # Step 3: Generate a random nonce and compute the commitment
    r = random.randint(0, q - 1)
    t = pow(g, r, p)

    # Step 4: Derive the challenge deterministically
    challenge_input = (str(t) + str(y)).encode()
    c = int.from_bytes(hashlib.sha256(challenge_input).digest(), byteorder='big') % q

    # Step 5: Compute the response
    s = (r + c * x) % q

    # Step 6: Verification
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p

    return lhs == rhs

p, q, g = generate_parameters() 
file_content = b"This is a confidential forensic file."

# Perform the proof
is_verified = schnorr_proof_of_possession(file_content, p, q, g)
print("Zero-Knowledge Proof of file possession verified:", is_verified)

Zero-Knowledge Proof of file possession verified: True


## Applying Schnorr in Digital Forensics

The Schnorr protocol is a widely used ZKP that can be applied in various digital forensics scenarios. Below is a simplified example of how the Schnorr protocol can be implemented to prove file possession without revealing the file itself.

In [19]:
def schnorr_prove(file_content, p, q, g, x):
    """Generate Schnorr proof for file possession."""
    h = int.from_bytes(hashlib.sha256(file_content).digest(), byteorder='big') % q  # Hash of the file
    r = random.randint(0, q - 1)  # Random nonce
    t = pow(g, r, p)              # Commitment
    c = int.from_bytes(hashlib.sha256((str(t) + str(h)).encode()).digest(), byteorder='big') % q  # Challenge
    s = (r + c * x) % q           # Response
    return {'commitment': t, 'challenge': c, 'response': s, 'public_value': pow(g, x, p)}

def schnorr_verify(proof, file_content, p, q, g):
    """Verify Schnorr proof."""
    h = int.from_bytes(hashlib.sha256(file_content).digest(), byteorder='big') % q  # Hash of the file
    t, c, s, y = proof['commitment'], proof['challenge'], proof['response'], proof['public_value']
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs

# Example usage
p, q, g = generate_parameters()
x, y = keygen(p, q, g)  # Generate keys
file_content = b"Confidential forensic evidence"
proof = schnorr_prove(file_content, p, q, g, x)  # Generate proof
is_valid = schnorr_verify(proof, file_content, p, q, g)  # Verify proof

print("Schnorr Proof:", proof)
print("Proof Valid:", is_valid)

Schnorr Proof: {'commitment': 4964, 'challenge': 22, 'response': 9, 'public_value': 2943}
Proof Valid: True


## Verifying a File

The following code demonstrates how Schnorr's protocol can be used to verify a file's integrity and detect tampering. The process involves:

1. **Generating an Initial File**: A file is created with a specific message.
2. **Generating a Proof**: A cryptographic proof is generated for the file using Schnorr's protocol.
3. **Verifying the Proof**: The proof is verified against the original file to confirm its integrity.
4. **Tampering with the File**: The file is modified to simulate tampering.
5. **Re-verifying the Proof**: The proof is verified again after tampering. This verification will fail, demonstrating the detection of tampering.

This workflow highlights the robustness of Schnorr's protocol in ensuring file integrity and detecting unauthorised modifications.

In [29]:
def schnorr_prove(file_content, p, q, g, x):
    h = int.from_bytes(hashlib.sha256(file_content).digest(), byteorder='big') % q
    r = random.randint(0, q-1)
    t = pow(g, r, p)
    c = int.from_bytes(hashlib.sha256((str(t) + str(h)).encode()).digest(), byteorder='big') % q
    s = (r + c * x) % q
    return {'commitment': t, 'challenge': c, 'response': s, 'public_value': pow(g, x, p), 'hash': h}

def schnorr_verify(proof, file_content, p, q, g):
    h = int.from_bytes(hashlib.sha256(file_content).digest(), byteorder='big') % q
    if h != proof['hash']:
        return False  # If file hash doesn't match the original, instantly fail
    t, c, s, y = proof['commitment'], proof['challenge'], proof['response'], proof['public_value']
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs

# Step 1: Generate initial file
file_name = "evidence.txt"
initial_message = "This is confidential forensic evidence."
with open(file_name, "w") as f:
    f.write(initial_message)

# Step 2: Generate parameters, key, and proof
p, q, g = generate_parameters()
x, _ = keygen(p, q, g)

with open(file_name, "rb") as f:
    original_content = f.read()

proof = schnorr_prove(original_content, p, q, g, x)

# Step 3: Verify original file
valid_before = schnorr_verify(proof, original_content, p, q, g)
print(f"Verification before tampering: {valid_before}")

# Step 4: Tamper with the file (add a word)
tampered_message = initial_message + " Altered."
with open(file_name, "w") as f:
    f.write(tampered_message)

with open(file_name, "rb") as f:
    tampered_content = f.read()

# Step 5: Reverify after tampering
valid_after = schnorr_verify(proof, tampered_content, p, q, g)
print(f"Verification after tampering: {valid_after}")

Verification before tampering: True
Verification after tampering: False
